In [199]:
# 直前のオッズリストをESに突っ込む
# http://odds.kyotei24.jp/
from bs4 import BeautifulSoup
import json
import urllib.request as u
import time
import re
import elasticsearch as es

import unicodedata

In [2]:
with open('../config/def-place.json','r',encoding='utf8') as defPlace:
    placeDict=json.load(defPlace)
print(placeDict[0]['placeId'],placeDict[0]['placeName'])
#for i in range(len(placeDict)):
#    print("|"+placeDict[i]['placeId']+"|"+placeDict[i]['placeName']+"|")

01 桐生


In [33]:
## 使わない。
def getRaceList(ymd):
    html = u.urlopen("http://odds.kyotei24.jp/index-%s.html" % ymd)
    time.sleep(1)
    soup = BeautifulSoup(html, "html.parser")
    for block in soup.select(".table.table-bordered"):
        idx = 0
        recs = block.select("tr")
        if len(recs) >= 1:
            kaijou = ''
            name = ''
            for rec in recs:
                tds = rec.select('td')
                if len(tds) > 1:
                    no = 0
                    for h in rec.select('a'):
                        no = h.string
                        kaijou=h['href'][14:16]
                        # todo Gradeが取れてない
                        gradepattern=r"【(.*)】"
                        grade=re.match(gradepattern , name)
                        detailUrl=h['href']
                        raceId = ymd+'-'+kaijou+'-'+no.zfill(2)
                        print(raceId,ymd,no,name,grade,kaijou,detailUrl)
                else:
                    head = tds.pop()
                    kaijou = head.b.string
                    name = head.contents[1].string


In [175]:
def getRaceListKoushiki(ymd):
    html = u.urlopen("https://www.boatrace.jp/owpc/pc/race/index?hd=%s" % ymd)
    time.sleep(1)
    soup = BeautifulSoup(html,"html.parser")
    for block in soup.select(".table1"):
        recs=block.select("tr")
        for rec in recs:
            tds=rec.select("td")
            if len(tds)>0:
                tmp=tds[0]
                #placePattern=r'tpl=(\d\d)'
                placePattern=r'tv(\d*)'
                placeId=re.search(placePattern,str(tmp)).group(1)
                #print(placeId)
                
                tmp=tds[2]
                gradePattern=r'class=\"is-(.*?)\"'
                grade=re.search(gradePattern,str(tmp)).group(1)
                #print(grade)
                
                rname=tds[4].string
                #print(rname)
                
                for i in range(1,12+1):
                    raceId=ymd+'-'+str(placeId).zfill(2)+'-'+str(i).zfill(2)
                    raceDate=ymd
                    raceNumber=i
                    raceName=rname
                    raceGrade=grade
                    racePlaceId=str(placeId).zfill(2)
                    
                    yield raceId,raceDate,raceNumber,raceName,raceGrade,racePlaceId

In [202]:
def getRaceMemberKoushiki(raceId):
    #html = u.urlopen("https://www.boatrace.jp/owpc/pc/race/index?hd=%s" % ymd)
    raceDate=raceId[0:8]
    raceNumber=int(raceId[12:14])
    placeId=raceId[9:11]
    print(raceDate,raceNumber,placeId)
    html = u.urlopen("https://www.boatrace.jp/owpc/pc/race/racelist?rno=%d&jcd=%s&hd=%s" %(raceNumber,placeId,raceDate) )
    time.sleep(1)
    soup = BeautifulSoup(html,"html.parser")
    for block in soup.select(".is-fs12"):
        #print(block)
        recs=block.select("tr")
        rec=recs[0]
        tds=rec.select("td")
        lane=unicodedata.normalize('NFKC', tds[0].string)        
        print(lane)
        
        print(tds[2])
        
raceId="20190321-01-01"
getRaceMemberKoushiki(raceId)

20190321 1 01
1
<td rowspan="4">
<div class="is-fs11">
                            3812 
                            / <span class="">B1</span>
</div>
<div class="is-fs18 is-fBold">
<a href="/owpc/pc/data/racersearch/profile?toban=3812">山崎　　聖司</a>
</div>
<div class="is-fs11">
                          東京/千葉
                          <br/>43歳/51.0kg
                        </div>
</td>
2
<td rowspan="4">
<div class="is-fs11">
                            3123 
                            / <span class="">B1</span>
</div>
<div class="is-fs18 is-fBold">
<a href="/owpc/pc/data/racersearch/profile?toban=3123">中西　　宏文</a>
</div>
<div class="is-fs11">
                          滋賀/滋賀
                          <br/>56歳/51.0kg
                        </div>
</td>
3
<td rowspan="4">
<div class="is-fs11">
                            3261 
                            / <span class="">B1</span>
</div>
<div class="is-fs18 is-fBold">
<a href="/owpc/pc/data/racersearch/profile?toban=3261">大川　　茂実</a>
</di

In [176]:
ymd = "20190321"
for ret in getRaceListKoushiki(ymd):
    print(ret)


('20190321-01-01', '20190321', 1, 'みどり市発足１３周年記念・市議会議長杯', 'ippan', '01')
('20190321-01-02', '20190321', 2, 'みどり市発足１３周年記念・市議会議長杯', 'ippan', '01')
('20190321-01-03', '20190321', 3, 'みどり市発足１３周年記念・市議会議長杯', 'ippan', '01')
('20190321-01-04', '20190321', 4, 'みどり市発足１３周年記念・市議会議長杯', 'ippan', '01')
('20190321-01-05', '20190321', 5, 'みどり市発足１３周年記念・市議会議長杯', 'ippan', '01')
('20190321-01-06', '20190321', 6, 'みどり市発足１３周年記念・市議会議長杯', 'ippan', '01')
('20190321-01-07', '20190321', 7, 'みどり市発足１３周年記念・市議会議長杯', 'ippan', '01')
('20190321-01-08', '20190321', 8, 'みどり市発足１３周年記念・市議会議長杯', 'ippan', '01')
('20190321-01-09', '20190321', 9, 'みどり市発足１３周年記念・市議会議長杯', 'ippan', '01')
('20190321-01-10', '20190321', 10, 'みどり市発足１３周年記念・市議会議長杯', 'ippan', '01')
('20190321-01-11', '20190321', 11, 'みどり市発足１３周年記念・市議会議長杯', 'ippan', '01')
('20190321-01-12', '20190321', 12, 'みどり市発足１３周年記念・市議会議長杯', 'ippan', '01')
('20190321-02-01', '20190321', 1, '第５４回ボートレースクラシック', 'SGa', '02')
('20190321-02-02', '20190321', 2, '第５４回ボートレースクラシック', 'SGa', '02')
(